import numpy

# Deep Learning from Scratch 2 : Chapter 04
# word2vec 속도 개선 (part 1)
## PyEmotion 발제 / YK Lee / 2021-06-03


앞의 3장에서 word2vec의 구조를 배우고 CBOW 모델을 구현했다. CBOW모델은 단순한 2층 신경망이라서 간단하게 구현 가능하다. 하지만 말뭉치에 포함된 어휘 수가 많아지면 계산량도 커진다는 것이 한계다. (시간이 너무 오래걸림)

이번 장의 목표는 word2vec의 속도 개선이다. 앞 장의 단순한 word2vec에서 2가지 개선을 추가할 것이다. 첫째, Embedding이라는 새로운 계층을 도입한다. 둘째, 네거티브 샘플링이라는 새로운 손실 함수를 도입한다. 이 두 가지 개선으로 우리는 '진짜' word2vec을 완성할 수 있다.
진짜 word2vec이 완성되면 PTB 데이터셋(실용적인 크기의 말뭉치)을 가지고 학습을 수행할 것이다. 그리고 그 결과로 얻은 단어의 분산 표현의 장점을 실제로 평가할 것이다.



# 4.1. word2vec 개선 1

앞 장의 복습부터 해보자. [그림 4-1]과 같은 CBOW 모델을 구현했었다.

[그림 4-1]과 같이 앞장의 CBOW모델은 단어 2개를 맥락으로 사용해, 이를 바탕으로 하나의 단어(타깃)을 추측한다. 

- 이때 입력 측 가중치(W<sub>in</sub>)와의 행렬 곱으로 은닉층이 계산된다. 다시 출력 - 측 가중치 (W<sub>out</sub>)와의 행렬 곱으로 각 단어의 점수를 구한다. 
- 그리고 이 점수에 Softmax 함수를 적용해 각 단어의 출현 확률을 얻는다. 
- 이 확률을 정답 레이블과 비교한다 (정확히는 교차 엔트로피 오차를 적용한다) => 손실을 구한다.


# 그림 4-1 
<img src = "./images_equations/fig 4-1.png" width=500>

_! warning_. 앞 장에서는 맥락의 윈도우 크기를 1로 한정함. 다시 말해 타깃 앞뒤 한 단어씩만 사용한 것임. 이번 장에서는 나중에 어떤 크기의 맥락도 다룰 수 있도록 기능을 추가할 것임

[그림 4-1]의 CBOW 모델도 작은 말뭉치를 다룰 때는 특별히 문제될 게 없다. 실제 [그림 4-1]에서 다루는 어휘는 모두 7개인데, 이 정도는 전혀 문제없이 처리할 수 있다. 그러나 거대한 말뭉치를 다루게 되면 몇 가지 문제가 발생한다. 그 문제점을 보여드리고자 어휘가 100만 개, 은닉층의 뉴런이 100개인 CBOW 모델을 생각해보자. 그러면 word2vec은 [그림4-2]처럼 된다. 

# 그림 4-2

<img src = "./images_equations/fig 4-2.png" width=500>

[그림 4-2]에서 보듯, 입력층과 출력층에는 각 100만 개의 뉴런이 존재한다. 이 수많은 뉴런 때문에 중간 계산에 많이 시간이 소요된다. 정확히는 다음의 두 계산이 **병목**이 된다. 

- 입력층의 원핫 표현과 가중치 행렬 W<sub>in</sub>의 계산(4.1에서 해결)
- 은닉층과 가중치 행렬 W<sub>out</sub>의 곱 및 Softmax 계층의 계산 (4.2에서 해결)

첫 번째는 입력층의 원핫 표현에 관한 문제다. 단어를 원핫 표현으로 다루기 때문에 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커지는 것이다. 예컨데 어휘가 100만 개라면 그 원핫 표현 하나만 해도 원소 수가 100만 개인 벡터가 된다 (즉, 상당한 메모리를 차지함). 이 원핫 벡터에 가중치 행렬 W<sub>in</sub>까지 곱하면 계산 자원/량이 더 가중된다.  =>  **Embedding 계층을 도입한다 (이번 4.1.절)**


두 번째는 은닉층 이후의 계산에 관한 문제다. 은닉층과 가중치 행렬 W<sub>out</sub>의 곱만 해도 계산량이 상당하다. 그리고 Softmax 계층에서도 다루는 어휘가 많아짐에 따라 계산량이 증가한다. =>  **Negative Sampling (다음 4.2절)으로 해결**

<노트>
개선 전의 word2vec 파일 (앞 장의 word2vec 구현)은 ch03 디렉터리의 ``` simple_cbow.py``` 혹은 ```simple_skip_gram.py```, 개선 후의 파일은 ch04 디렉터리의 ```cbow.py``` 혹은 ```skip_gram.py```에 있다. 



## 4.1.1. Embedding 계층

앞 장의 word2vec 구현에서는 단어를 원핫 표현으로 바꾼 후, MatMul 계층에 입력하고, MatMul 계층에서 가중치 행렬을 곱했다. 

그럼 여기서 어휘 수가 100만개인 경우를 상상해보자. 이때 은닉층 뉴런이 100개라면, MatMul 계층의 행렬 곱은 [그림4-3]처럼 된다. 

# 그림 4.3

<img src = "./images_equations/fig 4-3.png" width=500>

[그림 4-3]처럼 만약 100만개의 어휘를 담은 말뭉치가 있다면, 단어의 원핫 표현도 100만 차원이 된다 (그리고 이런 거대한 벡터와 가중치 행렬을 곱해야한다). 그러나 [그림4-3]에서 결과적으로 수행하는 것은 단지 **행렬의 특정 행을 추출**하는 것 뿐이다. 따라서 원핫 표현으로의 변환과 MatMul 계층의 행렬 곱 계산은 사실 필요가 없다. 

그러면 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는, Embedding 계층을 만들어보자. Embedding이란 단어는 단어 임베딩(word embedding)이라는 용어에서 유래했다. 즉, Embedding 계층에 단어 임베딩 (분산 표현)을 저장하는 것이다. 




# 4.1.2. Embedding 계층 구현

행렬에서 특정 행을 추출하는 건 아주 쉽다(!)

예컨데 가중치 W가 2차원 넘파이 배열일 때, 이 가중치로부터 특정 행을 추출하려면 그저 W[2]나 W[5]처럼 원하는 행을 명시하면 끝이다. 

파이썬 코드로 확인해보자


In [ ]:
import numpy as np

W = np.arange(21).reshape(7, 3) #넘파이로 행렬 만들기

W #만든 행렬 확인하기

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [ ]:
#3번째 행 추출
W[2]

array([6, 7, 8])

In [ ]:
#6번째 행 추출
W[5]

array([15, 16, 17])

또한, 가중치 W로부터 여러 행을 한꺼번에 추출하는 일도 간단하게 할 수 있다. 

원하는 행 번호들을 배열에 명시하기만 하면 된다.

파이썬 코드로 확인해보자

In [ ]:
idx = np.array([1, 0, 3, 0])

W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

이 예에서는 인덱스 4개(1, 0, 3, 0번째)를 한 번에 추출했다. 이처럼 인수에 배열을 사용하면 여러 행도 한꺼번에 추출할 수 있다. 참고로 이는 미니배치 처리를 가정했을 경우의 구현이다. 


그럼 Embedding 계층의 forward() 매서드를 구현하자. 지금까지의 예를 생각하면, 다음처럼 구현된다 (common/layers.py)



In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W] #이 책의 구현 규칙에 따라 인스턴스 변수 params와 grads를 사용한다. 
        self.grads = [np.zeros_like(W)] 
        self.idx = None  #인스턴스 변수 idx에는 추출하는 행의 인덱스(단어 ID)를 배열로 저장한다.

    def forward(self, idx):
        W, = self.params
        self.idx = idx 
        out = W[idx] 
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0 #가중치 기울기 dW를 꺼낸 다음, dW[...] = 0 문장에서 dW 원소를 0으로 덮어쓴다. dW 형상을 유지한 채, 그 원소를 0으로 덮어씀
        dW[self.idx] = dout #실은 나쁜 예. 앞 층에서 전해진 기울기 dout을 idx번째 행에 할당한다. 
        return None


여기서 역전파(backward)를 생각해보자. Embedding 계층의 순전파는 가중치 W의 특정 행을 추출할 뿐이었다. 단순히 가중치의 특정 행 뉴런만을 (아무것도 손대지 않고) 다음 층으로 흘러보낸 것이다. 

따라서 역전파에서는 앞 층 (출력 측 층)으로부터 전해진 기울기를 다음 층 (입력 측 층)으로 그대로 흘려주면 된다. 다만, 앞 층으로부터 전해진 기울기를 가중치 기울기 dW의 특정 행(idx번째 행)에 설정한다. 
그림으로는 [그림 4-4]처럼 된다. 

# 그림 4-4. Embedding계층의 forward와 backward 처리 (Embedding 계층은 Embed로 표기)]

<img src = "./images_equations/fig 4-4.png" width=500>

그런데 앞의 backward() 구현에는 사실 문제가 하나 있다. 바로 idx의 원소가 중복될 때 발생한다. 예컨데 idx가 [0, 2, 0, 4]일 경우를 생각해보자. 그렇다면 [그림 4-5]같은 문제가 생긴다. 

# 그림 4-5.

<img src = "./images_equations/fig 4-5.png" width=500>

[그림 4-5]와 같이 dh 의 각 행 값을 idx가 가리키는 장소에 할당해보겠다. 그러면 dW의 0번째 행에 2개의 값이 할당된다. 먼저 쓰여진 값을 덮어쓴다는 뜻이다. 

이 중복 문제를 해결하려면 '할당'이 아닌 '더하기'를 해야한다. 
즉, dh의 각 행의 값을 dW의 해당 행에 더해야한다. 
역전파를 다시 올바르게 구현하면 아래와 같다.

파이썬 코드로 확인해보자. 


In [ ]:
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0 #가중치 기울기 dW를 꺼낸 다음, dW[...] = 0 문장에서 dW 원소를 0으로 덮어쓴다. dW 형상을 유지한 채, 그 원소를 0으로 덮어씀
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]

    #혹은 numpy를 이용할 수 있다 
        #np.add.at(dW, self.idx, dout)
        
        return None

이렇게 word2vec (CBOW 모델)을 입력 측 MatMul 계층을 Embedding 계층으로 전환할 수 있다. 그 효과로 메모리 사용량을 줄이고 불필요한 계산도 생략할 수 있다. 

# 4.2. word2vec 개선 2 : 네거티브 샘플링

이제 남은 병목은 은닉층 이후의 처리 (행렬 곱과 Softmax 계층의 계산)이다. 이 병목을 **네거티브 샘플링**이라는 기법을 사용해서 해소하는 게 이번 절의 목표다. 

이번 절의 이야기는 특히 구현 쪽이 조금 복잡하다. 그래서 한 단계씩 천천히 확인하면서 진행할 것이다. 

# 4.2.1. 은닉층 이후 계산의 문제점

좀 더 이해를 돕기 위해 앞 절과 마찬가지로 어휘가 100만개, 은닉층 뉴런이 100개일때의 word2vec (CBOW 모델)을 예로 생각해보자. 이때 word2vec이 수행하는 작업은 [그림4-6]과 같다. 

# 그림 4-6

<img src = "./images_equations/fig 4-6.png" width = 300>

그림4-6에서 보듯, 입력층과 출력층에는 뉴런이 각 100만개씩 존재한다. 앞 절에서는 Embedding 계층을 도입하여 입력층 계산에서의 낭비를 줄였다. 남은 문제는 은닉층 이후의 처리다. 은닉층 이후에서 계산이 오래 걸리는 곳은 다음의 두 부분이다:

- 은닉층의 뉴런과 가중치 행렬(W<sub>out</sub>)의 곱
- Softmax 계층의 계산


첫 번째는 거대한 행렬을 곱하는 문제다. 앞의 예에서는 은닉층의 벡터 크기가 100이고, 가중치 행렬의 크기가 100 * 100만이다. 이렇게 큰 행렬의 곱을 계산하려면 시간이 오래걸린다. 그리고 메모리도 많이 필요하다. 또한, 역전파 때도 같은 계산을 수행하기 때문에 이 행렬 곱을 '가볍게' 만들어야 한다. 

두 번째로, Softmax에서도 같은 문제가 발생한다. 즉 어휘가 많아지면 Softmax의 계산량도 증가한다. 이 사실은 Softmax의 식을 보면 더욱 명확해진다. 


# 식 4.1 

<img src = "./images_equations/e 4-1.png" width=500>

[식 4.1]은 k번째 원소(단어)를 타겟으로 했을 때의 Softmax 계산식이다. 점수의 각 원소는 s<sub>1</sub>, s<sub>2</sub> ... ). 이 식에서는 어휘 수를 100만 개로 가정했으므로 분모의 값을 얻으려면 exp 계산을 100만번 수행해야 한다. 이 계산도 어휘 수에 비례해 증가하므로 Softmax를 대신할 '가벼운' 계산이 절실하다. 




# 4.2.2. 다중 분류에서 이진 분류로

지금부터 네거티브 샘플링 기법이 무엇인지 설명하겠다. 
본론부터 말하면 이 기법의 핵심 아이디어는 '이진 분류 binary classification'에 있다. 더 정확하게 말하면, **'다중 분류multi-classification (혹은 다중 클래스 분류)'**를  **'이진 분류'로 근사**하는 것이 네거티브 샘플링을 이해하는 데 중요하다. 

    - 이진분류는 Yes/No, 1/0 로 답하는 문제를 다룬다. "2는 짝수인가?" 같은 문제 등이 포함된다. 

### 다중분류 문제를 이진 분류 문제로 만들기 (근사)

지금까지는 맥락이 주어졌을 때 정답이 되는 단어를 높은 확률로 추측하도록 만들었다. 예컨데, 맥락으로 'you'와 'goodbye'를 주면 정답인 'say'의 확률이 높아지도록 신경망을 학습했다. 그리고 학습이 잘 이루어지면 그 신경망은 올바른 추측을 수행한다. 즉, 이 신경망은 "맥락이 'you'와 'goodbye'일 때, 타깃 단어는 무엇인가?" 라는 질문에 정답을 줄 수 있다. 

이제부터 우리가 생각해야할 것은, '다중 분류' 문제를 '이진 분류' 방식으로 해결하는 것이다. 예를 들어, "맥락이 'you'와 'goodbye'일 때, 타깃 단어는 'say'인가?"라는 질문에 답하는 신경망을 생각해야한다. 이렇게 하면 출력층에는 뉴런을 하나만 준비하면 된다. 출력층의 이 뉴런이 "say"의 점수를 출력하는 것이다. 

CBOW에서는 이 과정이 어떻게 이루어지는지 [그림4-7]을 보면 알 수 있다.  

# 그림 4-7

<img src = "./images_equations/fig 4-7.png" width=500>


[그림4-7]에서 보듯 출력층의 뉴런은 하나뿐이다. 따라서 은닉층과 출력 측의 가중치의 행렬의 내적은 "say"에 해당하는 열(단어 벡터)만을 추출하고, 그 추출된 벡터와 은닉층 뉴런과의 내적을 계산하면 끝이다. [그림 4-8]은 이 계산을 자세히 그린 것이다. 


# 그림 4-8

<img src = "./images_equations/fig 4-8.png" width=500>

[그림 4-8]처럼 출력 측의 가중치 W<sub>out</sub>에서는 각 단어 ID의 단어 벡터가 각각의 열로 저장되어 있다. 이 예에서는 "say"에 해당하는 단어 벡터를 추출한다. 그리고 그 벡터와 은닉층 뉴런과의 **내적**을 구한다. 이렇게 구한 값이 최종 점수이다. 

    이전까지의 출력층에서는 모든 단어를 대상으로 계산을 수행했다. 하지만 여기에서는 "say"라는 단어 하나에 주목하여 그 점수만을 계산하는 게 차이다. 그리고 시그모이드 함수를 이용해 그 점수를 확률로 변환한다. 

## 4.2.3. 시그모이드 함수와 교차 엔트로피 오차

이진 분류 문제를 신경망으로 풀려면 **1) 점수에 시그모이드 함수를 적용해 확률로 변환하고,** **2) 손실을 구할 때는 손실 함수로 '교차 엔트로피 오차'를 사용한다.** 이 둘은 이진 분류 신경망에서 가장 흔하게 사용하는 조합이다. 

    다중 분류의 경우, 출력층에는 (점수를 확률로 변환할 때) '소프트맥스 함수'를, 손실 함수로는 '교차 엔트로피 오차'를 이용한다. 이진 분류의 경우, 출력층에서는 '시그모이드 함수'를, 손실 함수로는 '교차 엔트로피 오차'를 이용한다. 


# 그림 4-9 

시그모이드 계층(왼쪽)과 시그모이드 함수의 그래프 (오른쪽)

<img src = "./images_equations/fig 4-9.png" width=500>


# 식 4.2 

시그모이드 함수

<img src = "./images_equations/e 4-2.png" width=500>


# 식 4.3. 

교차 엔트로피 오차

<img src = "./images_equations/e 4-2.png" width=500>


    이진 분류와 다중 분류 모두 손실 함수로 '교차 엔트로피 오차'를 사용한다. 각각의 수식은 [식4.3]과 [식1.7]로 서로 다르지만, 결국 의미는 같다. 정확히 말하면, 다중 분류에서 출력층에 뉴런을 2개만 사용할 경우 이진 분류의 [식4.3]과 완전히 같아진다. 따라서 Softmax with Loss 계층의 코드를 조금만 손보면 Sigmoid with Loss 계층도 구현 가능하다. 

# 그림 4-10. 

시그모이드 계층과 Cross Entropy Error 계층의 계산 그래프 (오른쪽은 Sigmoid with Loss 계층으로 통합한 모습) 

<img src = "./images_equations/fig 4-10.png" width=500>

[그림 4-10]에서 주목할 점은 역전파의 _y-t_ 값이다.* 여기에는 _y_는 신경망이 출력한 확률이고 _t_는 정답 레이블이다. 그리고 _y - t_ 는 정확히 그 두 값의 차이다. 예컨데 정답 레이블이 1이라면, y과 1(100%)에 가까워질 수록 오차가 줄어든다. 그리고 그 오차가 앞 계층으로 흘러가므로, 오차가 크면 '크게' 학습하고, 오차가 작으면 '작게' 학습하게 된다. 








## 4.2.4. 다중 분류에서 이진 분류로 (구현)

지금까지의 이야기를 구현 관점에서 정리해보자. 우리는 지금까지 다중 분류 문제를 다뤘다. 다중 분류에서는 출력층에 어휘 수만큼의 뉴런을 준비하고 이 뉴런들이 출력한 값을 Softmax 계층에 통과시켰다. 이때 이용되는 신경망을 '계층'과 '연산' 중심으로 그리면 [그림 4-11]처럼 된다. 


# 그림 4-11

<img src = "./images_equations/fig 4-11.png" width=500>


[그림 4-11]은 맥락이 "you"와 "goodbye"이고, 정답이 되는 타깃(예측해야 할 단어)이 "say"인 경우의 예이다 (단어 ID는 "you"가 0, "say"가 1, "goodbye"가 2라고 가정했다). 

또한, 입력층에서는 각각에 대응하는 단어 ID의 분산 표현을 추출하기 위해 Embedding 계층을 사용했다. 

- 앞 절에서는 Embedding 계층을 구현했다. 이 계층은 대상 단어 ID의 분산 표현(단어 벡터)을 추출한다. 이전에는 Embedding 계층 자리에 MatMul 계층을 사용했다. 

[그림 4-11]의 신경망을 이진 분류 신경망으로 변환해보자. 신경망 구성부터 보면 [그림 4-12]와 같다. 

# 그림 4-12 

<img src = "./images_equations/fig 4-12.png" width=500>


여기에는 은닉층 뉴런 _h_와, 출력 측의 가중치 W<sub>out</sub>에서 단어 "say"에 해당하는 단어 벡터와 내적을 계산한다.  그리고 그 출력을 Sigmoid with Loss 계층에 입력해 최종 손실을 얻는다. 

- [그림 4-12]에서는 Sigmoid with Loss 계층에 정답 레이블로 "1"을 입력한다. 이는 현재 문제의 답이 "Yes"임을 의미한다. 답이 "No"라면 Sigmoid with Loss에 정답 레이블로 "0"을 입력한다. 

앞으로의 이야기를 더 쉽게 풀어가기 위해 [그림 4-12]의 후반부를 더 단순하게 만들어보겠다. 이를 위해 Embedding Dot 계층을 도입한다. 이 계층은 [그림 4-12]의 Embedding 계층과 'dot 연산 (내적)'의 처리를 합친 계층이다. 이 계층을 사용하면 [그림 4-12]의 후반부를 [그림 4-13]처럼 그릴 수 있다. 

# 그림 4-13

<img src = "./images_equations/fig 4-13.png" width=500>


은닉층 뉴런 _h_는 Embedding Dot 계층을 거쳐 Sigmoid with Loss 계층을 통과한다. 보다시피 Embedding Dot 계층을 사용하면서 은닉층 이후의 처리가 간단해졌다. 


그럼 Embedding Dot 계층의 구현을 간단히 살펴보자
(ch04/negative_sampling_layer.py)


In [ ]:
class EmbeddingDot: #총 4개의 인스턴스 변수 (embed, params, grads, cache)
    def __init__(self, W):
        self.embed = Embedding(W) #Embedding계층 저장
        self.params = self.embed.params #이 책의 구현 규칙대로 매개변수 저장
        self.grads = self.embed.grads #기울기 저장
        self.cache = None  #순전파 시의 계산 결과를 잠시 유지하기 위한 변수로 사용
    
    def forward(self, h, idx): #순전파 담당하는 메서드. 인수로 은닉층 뉴런(h)과 단어 ID의 넘파이 배열(idx)을 받는다. 
        target_W = self.embed.forward(idx) #idx = 단어 ID의 '배열', 배열로 받는 이유는 데이터를 미니배치 처리로 가정했기 때문
        out = np.sum(target_W * h, axis = 1) #Embedding계층의 forward(idx)를 호출하고 내적 계산. 내적 계산은 np.sum()으로 가능

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W  = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh


# 내적 계산 구체적인 예

# 그림 4-14

<img src = "./images_equations/fig 4-14.png" width=500>


그림 4-14와 같이, 적당한 W와 h, 그리고 idx를 준비한다. 

- 여기서 idx가 [0, 3, 1]인데, 이는 3개의 데이터를 미니배치로 한 번에 처리하는 예임을 뜻한다. 

- idx가 [0, 3, 1]이므로 target_W는 W의 0 번, 3번, 1번째의 행을 추출한 결과다.

- target_W*h는 각 원소의 곱을 계산한다. 넘파이 배열의 * 연산은 원소별 곱을 수행한다. 

-  이 결과를 행마다 (axis = 1) 전부 더해 최종 결과 out을 얻는다. 

이상이 Embedding Dot 계층의 순전파다. 역전파는 순전파의 반대 순서로 기울기를 전달해 구현한다. 어렵지 않으니 여기선 역전파 구현 생략. 



## 4.2.5. 네거티브 샘플링

지금까지 배운 것으로 주어진 문제를 '다중 분류'에서 '이진 분류'로 변환 가능하다. 하지만 안타깝게도 이것만으로는 문제가 다 해결되지 않는다. 지금까지는 긍정적인 예 (정답)에 대해서만 학습했기 때문이다. 다시 말해 부정적인 예(오답)을 입력하면 어떤 결과가 나올지 확실하지 않다. 

앞의 예를 다시 한번 생각해보자. 앞의 예란 맥락이 "you"와 "goodbye"이고, 정답 타깃이 "say"인 경우다. 우리는 지금까지 긍정적인 예인 "say"만을 대상으로 이진 분류를 해왔다. 만약 여기서 '좋은 가중치'가 준비되어 있다면 Sigmoid계층의 출력(확률)은 1에 가까울 것이다. 이때의 처리를 계산 그래프로는 [그림 4-15]처럼 그릴 수 있다. 

# 그림 4-15. 

<img src = "./images_equations/fig 4-15.png" width=500>


현재의 신경망에서는 긍정적 예인 "say"에 대해서만 학습하게 된다. 그러나 부정적인 예 ("say"이외의 단어)에 대해서는 어떠한 지식도 획득하지 못했다. 여기에서 우리가 정말 하고 싶은 일은 무엇일까? 바로 긍정적 예("say")에 대해서는 Sigmoid계층의 출력을 1에 가깝게 만들고, 부정적 예("say"이외의 단어)에 대해서는 Sigmoid계층의 출력을 0에 가깝게 만드는 것이다. (그림 [4-16]참고)


# 그림 4-16

<img src = "./images_equations/fig 4-16.png" width=500>


예컨데, 맥락이 "you"와 "goodbye"일 때, 타깃이 "hello"일 확률 (틀린 단어일 경우의 확률)은 낮은 값이어야 바람직하다. [그림 4-16]에서 타깃이 "hello"일 확률은 0.021(2.1%)이다. 그리고 이런 결과를 만들어주는 **가중치**가 필요하다. 

    - 다중 분류 문제를 이진 분류로 다루려면 '정답'(긍정적인 예)'와 '오답(부정적인 예)' 각각에 대해 바르게 (이진) 분류할 수 있어야 한다. 따라서 긍정적 예와 부정적 예 모두를 대상으로 문제를 생각해야 한다. 

그러면 모든 부정적 예를 대상으로 하여 이진 분류를 학습시켜보면 어떨까? 답은 '아니오'다. 모든 부정적 예를 대상으로 하는 방법은 어휘 수가 늘어나면 감당할 수 없기 때문이다. ( 어휘 수가 증가에 대처하는 것이 이번 장의 목적인 만큼). 그래서 근사적인 해법으로, 부정적 예를 몇 개 (5개라든지, 10개라든지) 선택한다 (선택하는 방법은 뒤에서 설명). 

즉, 적은 수의 부정적 예를 샘플링해 사용한다. 이 것이 바로 네거티브 샘플링 기법이 의미하는 바이다. 

정리하면, 네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실을 구한다. 그와 동시에 부정적 예를 몇 개 샘플링(선별)하여, 그 부정적 예에 대해서도 마찬가지로 손실을 구한다. 그리고 각각의 데이터 (긍정적 예와 샘플링된 부정적 예)의 손실을 더한 값을 **최종 손실**로 한다. 

지금까지의 이야기를 구체적인 예로 살펴보자 . 물론, 지금까지와 같은 예 (긍정적 예의 타깃은 'say')를 다룬다. 그리고 부정적 예의 타깃을 2개 ('hello', 와 'I') 샘플링했다고 가정하자. 이제 CBOW 모델의 은닉층 이후만 주목하면 네거티브 샘플링의 계산 그래프는 [그림4-17]처럼 그릴 수 있다. 


# 그림 4-17

<img src = "./images_equations/fig 4-17.png" width=500>



[그림 4-17]에서 주의할 부분은 긍정적 예와 부정적 예를 다루는 방식이다 긍정적 예('say')에 대해서는 지금까지처럼 Sigmoid with Loss 계층에 정답 레이블로 "1"을 입력한다. 한편, 부정적 예("hello"와 "I")에 대해서는 Sigmoid with Loss 계층에 정답 레이블로 "0"을 입력한다. 그런 다음 각 데이터의 손실을 모두 더해 **최종 손실**을 출력한다. 

## 4.2.6. 네거티브 샘플링의 샘플링 기법

단순히 무작위로 샘플링하는 것보다 말뭉치의 통계 데이터를 기초로 샘플링 하는 것이 더 좋다. 

즉, 말뭉치에 자주 등장하는 단어를 많이 추출하고, 드물게 등장하는 단어를 적게 추출하는 것이다. 말뭉치에서의 단어 빈도를 기준으로 샘플링하려면, 먼저 말뭉치에서 각 단어의 출현 횟수를 구해 '확률분포'로 나타낸다. 그런 다음 그 확률분포대로 단어를 샘플링하면 된다 (그림 4-18)

확률분포대로 샘플링하므로 말뭉치에서 자주 등장하는 단어는 선택될 가능성이 높고, 같은 이유로 '희소한 단어'는 선택되기 어렵다. 

# 그림 4-18 확률분포에 따라 샘플링을 여러 번 수행한다.

<img src = "./images_equations/fig 4-18.png" width=500>


파이썬 코드로 확인해보자
(넘파이의 np.random.choice()메서드 활용 예 포함)


In [ ]:
import numpy as np

#0부터 9까지의 숫자 중 하나를 무작위로 샘플링
np.random.choice(10)

1

In [ ]:
np.random.choice(10)

3

In [ ]:
#words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'goodbye'

In [ ]:
#5개만 무작위로 샘플링 (중복 있음)

np.random.choice(words, size = 5) #인수로 size를 지정하면 샘플링을 size만큼 수행한다. 


array(['say', 'you', 'say', 'say', '.'], dtype='<U7')

In [ ]:
#5개만 무작위로 샘플링 (중복 없음)

np.random.choice(words, size = 5, replace = False) #인수에 replace =False를 하면 샘플링시 중복을 없애준다.


array(['goodbye', 'you', 'I', 'say', '.'], dtype='<U7')

In [ ]:
#확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1] #p에 확률분포를 담은 리스트를 지정하면 그 확률 분포대로 샘플링한다. 

np.random.choice(words, p=p)

'say'

! 한 가지 더.

word2vec의 네거티브 샘플링에서는 앞의 확률분포에서 한 가지를 수정하라고 권고하고 있다. 바로 [식 4.4]처럼 기본 확률분포에 0.75를 제곱하는 것이다. 

식 4.4

<img src = "./images_equations/e 4-4.png" width=500>


여기서 P(_w_<sub>i</sub>)는 i번째 단어의 확률을 뜻한다. 
[식 4.4]는 단순히 원래 확률분포의 각 요소를 '0.75 제곱'할 뿐이다. 다만 수정 후에도 확률의 총합은 1이 되어야 하므로, 분모로는 '수정 후 확률분포의 총합'이 필요하다. 

그런데 [식 4.4]처럼 수정하는 이유는 무엇일까? 그것은 출현 확률이 낮은 단어를 '버리지 않기' 위해서다. ('0.75 제곱'을 함으로써, 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있다.
아래 예처럼 말이다

In [ ]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


예시에서 보듯 수정 전 확률이 0.01(1%)이던 원소가, 수정 후에는 0.0265..(약 2.65%)로 높아진 것을 확인할 수 있다.

이처럼 낮은 확률 단어가 (조금 더) 쉽게 샘플링되도록 하기 위한 구제 조치로써 "0.75 제곱"을 수행한다. 참고로 0.75라는 수치에는 이론적인 의미는 없으니 다른 값으로 설정해도 된다. 

지금까지 살펴본 것처럼 네거티브 샘플링은 말뭉치에서 단어의 확률분포를 만들고, 다시 0.75를 제곱한 다음, np.random.choice()를 사용해 부정적 예를 샘플링한다. 

이 책에서는 이 처리를 담당하는 클래스를 ```UnigramSampler```라는 이름으로 제공한다. 여기에서는 ```UnigramSampler```의 사용법만 간단히 설명할테니 구현에 관심있는 분은 Ch04/negatvie_sampling_layer.py를 참고할 것. 



[옵션] ```UnigramSampler``` 구현

```UnigramSampler``` 클래스는 초기화 시 3개의 인수를 받는다. 바로 단어 ID 목록인 corpus, 확률분포에 '제곱'할 값인 power(기본값은 0.75), '부정적 예 샘플링'을 수행하는 횟수인 ```sample_size```이다. 

또한 ```UnigramSampler``` 클래스는 ```get_negative_sample(target)``` 메서드를 제공한다. 이 메서드는 target 인수로 지정한 단어를 긍정적 예로 해석하고, 그 이외의 단어 ID를 샘플링한다 (즉, 부정적 예를 골라준다). 

파이썬으로 예시를 확인해보자



In [ ]:
import collections

In [ ]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample


In [ ]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])

power = 0.75

sample_size = 2


sampler = UnigramSampler(corpus, power, sample_size)

target = np.array([1, 3, 0])

negative_sample = sampler.get_negative_sample(target)

print(negative_sample)

```

 [[0 3]
  [1 2]
  [2 3]]

```

여기서는 긍정적 예로 [1, 3, 0]이라는 3개의 데이터를 미니배치로 다뤘다. 이 각각의 데이터에 대해서 부정적 예를 2개씩 샘플링하겠다. 이 예에서는 첫 번째 데이터에 대한 부정적 예는 [0 3], 두 번째는 [1 2], 3번째는 [2 3]이 뽑혔음을 알 수 있다. (실행할 때마다 결과가 달라질 수 있다).

이제 우리는 부정적 예를 샘플링할 수 있게 되었다. 

## 4.2.7. 네거티브 샘플링 구현

마지막으로 네거티브 샘플링을 구현해보겠다. 

```NegativeSamplingLoss``` 라는 클래스로 구현해보자. 

우선은 초기화 메서드이다 (ch04/negative_sampling_layer.py)


In [ ]:
#초기화 메서드 인수: 출력 측 가중치 W, 말뭉치(단어의 ID리스트) corpus, 확률분포에 제곱할 power, 네거티브 샘플링 횟수 sample
# 앞 절에서 설명한 UnigramSampler 클래스를 생성 후 인스턴스 변수인 sampler로 저장한다.
#네거티브 샘플링 횟수는 인스턴스 변수인 sample_size에 저장한다

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5): 
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] #원하는 계층을 리스트로 보관 
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] #원하는 계층을 리스트로 보관
        #부정적 예를 다루는 계층 sample_size개수 + 긍정적 예를 다루는 계층 1개 (0번째 계층 loss layer[0], embed_dot_layers[0])

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target): # 은닉층 뉴런 h와 긍정적 예 타깃 target을 인수로 받음
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target) #부정적 예를 샘플링한 후 여기에 저장
       
       # 그 다음, 긍정적 예와 부정적 예 각각의 데이터에 대해 순전파를 수행해 그 손실들을 더한다.
         #구체적으로는 Embedding Dot 계층의 forward 점수를 구하고, 
         # 이어서 이 점수와 레이블을 Sigmoid with Loss 계층으로 흘려 손실을 구한다

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32) #긍정적 예의 정답 레이블
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32) #부정적 예의 정답 레이블
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

#역전파
#순전파 때의 역순으로 각 계층의 backward()를 호출
#은닉층의 뉴런은 순전파 시에 여러개로 복사됨 (1.3.4. 계산 그래프 절의 Repeat노드) 
# 따라서 역전파 때는 여러 개의 기울기 값을 더해준다. 

    def backward(self, dout=1): 
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

# 4.3. 개선판 word2vec학습

앞에서 개선한 2개를 신경망 구현에 적용해보자. 그런 다음 PTB데이터셋을 사용해 학습하고 더 실용적인 단어의 분산 표현을 얻어보자

## 4.3.1. CBOW 모델 구현


In [3]:
!pwd

/Users/yklee/pyemotion_understand


In [4]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW: #초기화 메서드는 4개의 인수를 받는다 (어휘수, 은닉층의 뉴런수, 맥락의 크기ㅣ (주변 단어 중 몇 개나 맥락으로 포함할지), 단어 id목록)
    def __init__(self, vocab_size, hidden_size, window_size, corpus): #맥락=2이며, 타깃 단어의 좌우 2개씩, 총 4개 단어.
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):#앞 장에서는 단어 아이디를 원핫 벡터로 변환해서 사용했었다. 여기선 이 부분만 다름)
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


## 그림 4-19 맥락과 타깃을 단어 ID로 나타낸 예 (맥락의 윈도우 크기는 1)

<img src = "./images_equations/fig 4-19.png" width=500>



## 4.3.2. CBOW모델 학습 코드

마지막으로 CBOW 모델의 학습을 구현하자. 여기서는 단순히 신경망 학습을 수행할 뿐이다. 

이번 CBOW 모델은 윈도우 크기를 5로, 은닉층의 뉴런 수를 100개로 설정했다. 사용하는 말뭉치에 따라 다르지만 윈도우 크기는 2~10개, 은닉층의 뉴런 수 (단어의 분산 표현의 차원 수)는 50~500개 정도면 좋은 결과를 얻을 수 있다. 이러한 하이퍼파라미터에 관한 논의는 잠시 후에. 

이번에 다루는 PTB는 지금까지의 말뭉치보다 월등히 커서 학습 시간이 상당히 오래 걸린다 (반나절 정도). 그래서 GPU를 사용할 수 있는 모드를 준비했다. GPU로 실행하려면 파일 앞부분에 있는 #config.GPU = True 주석을 헤제하면 된다. (단, GPU로 실행하려면 엔비디아 GPU를 장착한 컴퓨터여야 하고 쿠파이도 미리 설치해야함)

학습이 끝나면 가중치를 꺼내 (여기에서는 입력 측 가중치만), 나중에 이용할 수 있도록 파일에 보관한다 (단어와 단어 ID변환을 위해 사전도 함께 보관). 파일로 저장할 때는 파이썬의 피클(pickle) 기능을 이용한다. 
피클은 파이썬 코드의 객체를 파일로 저장 (또는 파일에서 읽기) 하는 데 유용하다. 

***
여기선 이미 학습이 완료된 매개변수를 ch04/cbow_params.pkl에 준비했다. 학습이 끝나기를 기다리기 지루하다면 이 책에서 제공하는 학습이 완료된 매개변수를 이용하면 된다. 참고로 학습된 가중치 데이터는 각자가 학습한 환경에 따라 다르다.학습 과정에서 무작위 값이 쓰이기 때문인데, 예컨데 가중치 초기화에 이용하는 초깃값 미니배치 선택, 네거티브 샘플링에서 샘플링이 무작위로 이뤄진다. 이러한 무작위성 때문에 최종적으로 얻는 가중치가 각자의 환경에 따라 달라지지만, 거시적으로 보면 비슷한 결과 (경향)를 얻을 수 있다. 


코드를 살펴보자 (ch04/train.py)

In [ ]:
### GPU가 있을때만 실행해볼 것, 생략 가능 (다음 코드에서 pkl불러오기)

# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


## 4.3.3. CBOW 모델 평가

그러면 앞 절에서 학습한 단어의 분산 표현을 평가해보자. 2장에서 구현한 most_similar()메서드를 이용하여, 단어 몇 개에 대해 거리가 가장 가까운 단어들을 뽑아보자 (ch04/eval.py)

In [6]:
!pwd

/Users/yklee/pyemotion_understand


In [7]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'ch04/cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)




[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 가까이 모을 뿐 아니라, 더 복잡한 패턴을 파악하는 것으로 알려져 있다. 대표적인 예가 "king-man+woman=queen"으로 유명한 유추 문제 (비유 문제)이다. 더 정확하게 말하면, word2vec의 단어의 분산 표현을 사용하면 유추 문제를 벡터의 덧셈과 뺄셈으로 풀 수 있다는 뜻이다. 

실제로 유추 문제를 풀려면 [그림 4-20]처럼 단어 벡터 공간에서 "man->woman"벡터와 "king->?"벡터가 가능한 한 가까워지는 단어를 찾는다.

## [그림4-20] "man: woman = king: ?" 유추 문제 풀기 (단어 벡터 공간에서 각 단어의 관계성)
<img src = "./images_equations/fig 4-20.png" width=500>


단어 'man'의 분산 표현(단어 벡터)을 'vec('man')'이라고 표현해보자. 그러면 그림4-20에서 얻고 싶은 관계를 수식으로 나타내면 vec('woman') - vec('man') = vec(?) - vec('king')이 된다. 즉, 우리가 풀어야하는 문제는 'vec('king') + vec('woman') - vec('man') = vec(?)'라는 벡터에 가장 가까운 단어 벡터를 구하는 일이 된다. 이 로직을 구현한 함수는 common/util.py 파일의 analogy()이다. 이 함수를 사용하면 지금과 같은 유추 문제를 analogy('man', 'king', 'woman', word_to_id, id_to_word, word_vecs, top=5)라는 한 줄로 처리할 수 있다. 이 함수를 실행하면 결과가 다음과 같은 형태로 출력된다.



In [8]:
# 유추(analogy) 작업
#첫번째 줄에 문제 문장이 출력, 다음 줄 부터는 점수가 높은 순으로 5개의 단어가 출력된다 (그 옆에는 점수)
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875


Note* 두 번째 문제: go, went라고 답한 것을 보면 현재형과 과거형 패턴을 파악하고 있다는 증거다. 시제 정보가 단어의 분산 표현에 인코딩되고 있음을 알 수 있다. 세 번째에서도 단수형과 복수형을 바르게 파악했다. 네 번째는 bad, worse라고 답하지 않았다. 하지만 more와 less등의 비교급 단어를 제시한 걸 보면, 비교급이라는 성질도 단어의 분산 표현에 인코딩되어 있음을 알 수 있다.

In [9]:
print('-'*50)
analogy('student', 'book', 'teacher',  word_to_id, id_to_word, word_vecs)


--------------------------------------------------

[analogy] student:book = teacher:?
 woman: 4.1796875
 street: 4.171875
 businessland: 3.8046875
 wife: 3.76171875
 judge: 3.66015625


In [14]:
analogy('korea', 'korean', 'usa',  word_to_id, id_to_word, word_vecs)



[analogy] korea:korean = usa:?
 other: 5.02734375
 inc: 4.75390625
 said: 4.421875
 agreed: 3.931640625
 new: 3.9140625


In [15]:
print('-'*50)
analogy('paper', 'papers', 'book',  word_to_id, id_to_word, word_vecs)


--------------------------------------------------

[analogy] paper:papers = book:?
 a.m: 4.3828125
 partnership: 4.33203125
 veto: 4.04296875
 second: 3.919921875
 request: 3.765625


# 4.4. word2vec 남은 주제

## 4.4.1. word2vec을 사용한 애플리케이션 예

- word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 찾는 용도로 쓸 수 있다.
- 그 외에도, 전이 학습이 가능하단 점에서 유용하다 (자연어처리에서 단어의 분산 표현이 중요한 이유). 전이 학습 = 한 분야에서 배운 지식을 다른 분야에도 적용하는 기법
- 단어의 분산 표현 = 단어를 고정 길이 벡터로 변환해줌(장점). 문장 (단어의 흐름)도 단어의 분산 표현을 사용해 고정 길이 벡터로 변환 가능. 다음 5장에서 설명할 순환 신경망(RNN)을 사용하면 한층 세련된 방법으로 (word2vec의 단어의 분산 표현을 이용하면서) 문장을 고정 길이 벡터로 변환 가능하다. 
- 단어나 문장을 고정 길이 벡터로 변환할 수 있다는 점은 매우 중요 = 자연어를 벡터로 변환해서 일반적인 머신러닝 기법(신경망이나 svm 등)을 적용할 수 있기 때문이다. (그림 4-21)

<img src = "./images_equations/fig 4-21.png" width=500/>

<img src = "./images_equations/fig 4-22.png" width=500/>

## 4.4.2. 단어 벡터 평가 방법

word2vec을 통해 얻은 단어의 분산 표현이 좋은지 어떻게 평가할까?

자주 사용되는 평가 척도는 단어의 '유사성'이나 '유추 문제'를 활용한 평가

단어의 유사성 평가:
- 사람이 작성한 단어 유사도를 검증 세트로 사용해 평가
- 유사도를 0에서 10사이로 점수화한다면, 'cat'과 'animal' 의 유사도는 8점, 'cat'과 'car'의 유사도는 2점과 같이, 사람이 단어 사이의 유사한 정도를 규정한다.
- 사람이 부여한 점수와 word2vec에 의한 코사인 유사도 점수를 비교해 그 상관성을 본다

유추 문제를 활용한 평가:
- "king: queen = man : ?" 와 같은 유추 문제를 출제하고, 그 정답률로 단어의 분산 표현의 우수성을 측정한다. 예컨대 논문 [27]에는 유추 문제에 의한 평가 결과가 실려 있는데, [그림4-23]은 그중 일부를 발췌한 것이다.

- 유추 문제를 통해 '단어의 의미나 문법적인 문제를 제대로 이해하고 있는지'를 (어느 정도)측정할 수 있다. 그러므로 유추 문제를 정확하게 풀 수 있는 단어의 분산 표현이라면 자연어를 다루는 애플리케이션에 얼마나 기여하는지 (혹은 기여하지 않는지)는 애플리케이션 종류나 말뭉치의 내용 등, 다루는 문제 상황에 따라 다르다. 즉, 유추 문제에 의한 평가가 높다고 해서 여러분의 애플리케이션에서도 반드시 좋은 결과가 나오리라는 보장은 없으니 이 점을 주의하라.

##[그림4-23] 유추 문제에 의한 단어 벡터의 평가 결과 (논문 [27]에서 발췌)
<img src = "./images_equations/fig 4-23.png" width=500>

Glove - Pennington et al 2014; Stanford

그림 4-23 결과로부터 다음 사항을 알 수 있다:

- 모델에 따라 정확도가 다르다 (말뭉치에 따라 적합한 모델 선택)
- 일반적으로 말뭉치가 클수록 결과가 좋다 (항상 데이터가 많은 게 좋음)
- 단어 벡터 차원 수는 적당한 크기가 좋다 (너무 커도 정확도가 나빠짐) 

# 정리

- 이 장의 핵심은 '모두' 대신 '일부'를 처리하는 것이다. 인간 역시 모든 것을 알 수 없듯이 컴퓨터도 현재의 성능으로는 모든 데이터를 처리하는 것은 비현실적이다. 그 보다는 꼭 필요한 이룹에 집중하는 편이 얻는 게 많다. 

- 이번 장에서는 이 생각에 기초한 기법인 네거티브 샘플링을 자세하게 살펴보았다. 네거티브 샘플링은 '모든' 단어가 아닌 '일부' 단어만을 대상으로 하는 것으로, 계산을 효율적으로 수행해준다.